In [143]:
import auxiliary_functions as aux
import time
from classification import train_randomforest_AUC
from filters import runAutoFilter
from filters import apply_chosenfilterstrategy
from numpy import median
from sklearn.model_selection import StratifiedKFold  # maintains class proportions when creating folds



from filters import getscores_decisionstumpfilter

# Creates a train and test set for each fold of a stratified kfold process,
# then calls the train_classifier function and prints the average results
def runClassificationExperiment(df, apply_undersampling, unusable_feature_threshold, filter_method, nfeatures):
    kf = StratifiedKFold(n_splits=10, shuffle=False)
    score_array = []
    fold = 0
    for train_index, test_index in kf.split(df.iloc[:, :-1], df.iloc[:, -1]):
        fold = fold + 1
        training_set = df.iloc[train_index, :]
        test_set = df.iloc[test_index, :]
        training_set, test_set = aux.removeLowFrequencyFeatures_TrainTest(training_set, test_set, unusable_feature_threshold)
        
        # count features
        print(f'Features: {training_set.shape[1]}')

        if filter_method == "None":
            X_train, X_test = training_set.iloc[:, :-1], test_set.iloc[:, :-1]
        elif filter_method == "AutoFilter":
            # nfeatures_array = aux.set_nfeatures_array(df.shape[1])  # k values to be used (GenAge dataset)
            nfeatures_array = aux.set_nfeatures_array(training_set.shape[1])  # k values to be used (GenAge dataset)
            filter_method, nfeatures = runAutoFilter(training_set, apply_undersampling, nfeatures_array)
            print(f'AutoFilter InternalCV end. Selected filter strategy : {filter_method} with k = {nfeatures}')
            X_train, X_test = apply_chosenfilterstrategy(df, train_index, test_index, filter_method, nfeatures)
        else:  # other filter methods
            X_train, X_test = apply_chosenfilterstrategy(df, train_index, test_index, filter_method, nfeatures)

        y_train, y_test = training_set.iloc[:, -1], test_set.iloc[:, -1]

        #Training classifier - AUC Scoring
        score = train_randomforest_AUC(X_train, X_test, y_train, y_test, apply_undersampling)
        #score = train_randomforest_gmean(X_train, X_test, y_train, y_test, apply_undersampling) #for GMean scoring
        score_array.append(score)
        print(f"Fold {fold}. AUC score: {round(score, 3)}")

    print(f"Median AUC: {round(median(score_array),3)}")


"""
Main function: runs a 10-fold cross-validation and returns the AUC results of Random Forest classifiers trained at each fold, and the median AUC value.
The input datasets should be formatted as tab-separated spreadsheets with an index column as the first (leftmost) variable, and a binary (0,1) class value in the last (rightmost) position
For our test datasets, there are some additional identifier variables which are removed when loading the dataset: STITCH_Code, InteractorsList, InteractorsCount
The following experiment parameters that can be set by the user:

unusable_feature_threshold: 
    Features with too few different values are removed in a preprocessing step, removing spurious variables
    This works well for binary features, but is not recommended for features with numerical or categorical values
    int value (recommended for our test datasets: 10). If set to a value < 0, the threshold filter is not applying
    
BRF_undersampling:
    Whether the classifiers should be trained using balanced datasets to avoid a bias in favour of the majority class
    True: undersample training set to a 1:1 ratio (uses BalancedRandomForestClassifier from imblearn.ensemble library).
    False: trains classifier with unchanged class balance (Not recommended for imbalanced datasets)
    
filter_method: 
    Selected filter scoring strategy. Accepted values: 
    "None": Do not apply a filter to the dataset, and use all predictive features when training the classifier
    "AutoFilter": Select a candidate filter and k value by testing all combinations in an internal cross-validation
    "InfoGain": Apply Information Gain filter (uses native implementation from sklearn)
    "Chi2": Apply Chi2 filter (uses native implementation from sklearn)
    "DecisionStump": Apply Decision Stump filter (implemented in this project)
    "LogOddsRatio": Apply Log Odds Ratio filter (implemented in this project)
    "AsymmetricOptimalPrediction": Apply Asymmetric Optimal Prediction filter (implemented in this project)
    
nfeatures: number of predictive features to be selected when applying the filter method
    int value, irrelevant for filter_method = None (no filter) and for filter_method = AutoFilter (chosen automatically)    
"""
def main(filepath, BRF_undersampling, unusable_feature_threshold, filter_method, nfeatures):
    if filepath == "":
        df = aux.load_dataset_dialog()
    else:
        df = aux.load_dataset_filepath(filepath)
    df = aux.removeLowFrequencyFeatures(df, unusable_feature_threshold)  # Apply threshold filter to full dataset (reduce runtime)
    runClassificationExperiment(df, BRF_undersampling, unusable_feature_threshold, filter_method, nfeatures)
    


In [144]:
filenames = [
    'C Elegans dataset examples\Version-1 datasets (no score threshold)\CElegans Protein Interactors dataset_v1.tsv',
    'C Elegans dataset examples\Version-1 datasets (no score threshold)\CElegans GOTerms dataset_v1.tsv',
    # 'C Elegans dataset examples\Version-1 datasets (no score threshold)\CElegans Phenotypes dataset_v1.tsv',
    # 'C Elegans dataset examples\Version-1 datasets (no score threshold)\CElegans GenAge dataset_v1.tsv',
    # 'C Elegans dataset examples\Version-2 datasets (45% minimum score threshold)\CElegans Protein Interactors dataset_v2.tsv',
    # 'C Elegans dataset examples\Version-2 datasets (45% minimum score threshold)\CElegans GOTerms dataset_v2.tsv',
    # 'C Elegans dataset examples\Version-2 datasets (45% minimum score threshold)\CElegans Phenotypes dataset_v2.tsv',
    # 'C Elegans dataset examples\Version-2 datasets (45% minimum score threshold)\CElegans GenAge dataset_v2.tsv',
    # 'Drosophila datasets (validation experiment)\Drosophila Protein Interactors .tsv',
    # 'Drosophila datasets (validation experiment)\Drosophila GO Terms.tsv',
]

filters = [
    # "None",
    # "AutoFilter",
    "InfoGain",
    # "Chi2",
    # "DecisionStump",
    # "LogOddsRatio",
    # "AsymmetricOptimalPrediction",
]

# Test parameters
# filename = 'C Elegans dataset examples\Version-1 datasets (no score threshold)\CElegans GOTerms dataset_v1.tsv' # set as empty ("") to have a file selection dialog
#filename = ""  # set as empty ("") to have a file selection dialog


In [106]:
df = aux.load_dataset_dialog()

In [108]:
getscores_decisionstumpfilter(df)

{'GO:0099503': 0.43907967811402016,
 'GO:0007606': 0.4390804597701149,
 'GO:0031528': 0.4391930182477615,
 'GO:0046058': 0.44497336506857077,
 'GO:0019933': 0.44786353847897536,
 'GO:0098793': 0.448996939816389,
 'GO:0040013': 0.4519437832936643,
 'GO:0007611': 0.4552873172390344,
 'GO:0007631': 0.4552873563218391,
 'GO:0046662': 0.4560240281083532,
 'GO:0040017': 0.4595375722543353,
 'GO:0043059': 0.4596551724137931,
 'GO:0031683': 0.462937776266576,
 'GO:0010226': 0.4633911368015414,
 'GO:0042277': 0.46413793103448275,
 'GO:0050805': 0.4671313612150063,
 'GO:0004016': 0.46871812308738525,
 'GO:0006171': 0.46871812308738525,
 'GO:0005254': 0.4690014734217387,
 'GO:0005253': 0.4690014734217387,
 'GO:0050890': 0.4697381842910575,
 'GO:0032222': 0.470531565227247,
 'GO:0071495': 0.4706896551724138,
 'GO:0032370': 0.47114942528735626,
 'GO:0032368': 0.47114942528735626,
 'GO:0004058': 0.47245834750085003,
 'GO:0036468': 0.47245834750085003,
 'GO:0007166': 0.4727416978352034,
 'GO:0043949'

In [145]:
for filter_ in filters:
    for filename in filenames:
        unusable_feature_threshold = 10  # features with too few different values are removed in a preprocessing step
        BRF_undersampling = False  # True: undersample the training set to a 1:1 ratio (uses BRF for RF classifiers).
        filter_method = filter_
        nfeatures = 0
        print(f'Opening the file: {filename}, for filter: {filter_}')
        start_time = time.time()
        main(filename, BRF_undersampling, unusable_feature_threshold, filter_method, nfeatures)  # runs main function
        end_time = time.time()
        print(f'total runtime: {int(end_time-start_time)}s\n\n')

Opening the file: C Elegans dataset examples\Version-1 datasets (no score threshold)\CElegans Protein Interactors dataset_v1.tsv, for filter: InfoGain
Features: 5303
Warning, invalid k value. k: 0 features: 5789. Setting k as all.
Fold 1. AUC score: 0.813
Features: 5562
Warning, invalid k value. k: 0 features: 5789. Setting k as all.
Fold 2. AUC score: 0.818
Features: 5518
Warning, invalid k value. k: 0 features: 5789. Setting k as all.
Fold 3. AUC score: 0.57
Features: 5558
Warning, invalid k value. k: 0 features: 5789. Setting k as all.
Fold 4. AUC score: 0.749
Features: 5678
Warning, invalid k value. k: 0 features: 5789. Setting k as all.
Fold 5. AUC score: 0.581
Features: 5607
Warning, invalid k value. k: 0 features: 5789. Setting k as all.
Fold 6. AUC score: 0.719
Features: 5571
Warning, invalid k value. k: 0 features: 5789. Setting k as all.
Fold 7. AUC score: 0.87
Features: 5532
Warning, invalid k value. k: 0 features: 5789. Setting k as all.
Fold 8. AUC score: 0.652
Features: 5

In [1]:
import pandas as pd
import numpy as np
import scipy

cego = pd.read_csv('C Elegans dataset examples\Version-1 datasets (no score threshold)\CElegans GOTerms dataset_v1.tsv', sep='\t')
cepi = pd.read_csv('C Elegans dataset examples\Version-1 datasets (no score threshold)\CElegans Protein Interactors dataset_v1.tsv', sep='\t')
drgo = pd.read_csv('Drosophila datasets (validation experiment)\Drosophila GO Terms.tsv', sep='\t')
drpi = pd.read_csv('Drosophila datasets (validation experiment)/Drosophila Protein Interactors .tsv', sep='\t')


In [2]:
from sklearn.model_selection import StratifiedKFold  # maintains class proportions when creating folds
def markFeatureAsLowFrequency(training_set, threshold):
    features = []
    if threshold > 0:
        number_instances = training_set.shape[0]
        for feature in training_set:
            try:
                mostFrequent = training_set[feature].value_counts(ascending=True)[0]
            except:
                mostFrequent = 0
            if number_instances - mostFrequent < threshold:
                features.append(feature)
    return features
def getFilteredFeatures(df, unusable_feature_threshold):
    kf = StratifiedKFold(n_splits=10, shuffle=False)
    fold = 0
    allFoldFeatures = []
    for train_index, test_index in kf.split(df.iloc[:, :-1], df.iloc[:, -1]):
        fold = fold + 1
        training_set = df.iloc[train_index, :]
        lowFrequency = markFeatureAsLowFrequency(training_set, unusable_feature_threshold)
        for low in lowFrequency:
            if not (low in allFoldFeatures):
                allFoldFeatures.append(low)
        
        print(f'{fold-1}, {len(lowFrequency)}, {len(allFoldFeatures)}')
    return allFoldFeatures



In [5]:
filtered_go = getFilteredFeatures(cego, 10)

0, 1296, 1296
1, 1306, 1308
2, 1333, 1342
3, 1325, 1367
4, 1357, 1400
5, 1345, 1412
6, 1381, 1442
7, 1414, 1486
8, 1484, 1549
9, 1884, 1913



k | drop | cummulative
--- | --- | ---
0 | 1296 | 1296
1 | 1306 | 1308
2 | 1333 | 1342
3 | 1325 | 1367
4 | 1357 | 1400
5 | 1345 | 1412
6 | 1381 | 1442
7 | 1414 | 1486
8 | 1484 | 1549
9 | 1884 | 1913

In [6]:
filtered_pi = getFilteredFeatures(cepi, 10)

0, 486, 486
1, 227, 535
2, 271, 658
3, 231, 699
4, 111, 703
5, 182, 715
6, 218, 730
7, 257, 751
8, 369, 791
9, 196, 799


In [7]:
len(filtered_go), len(filtered_pi)

(1913, 799)

In [8]:

remove = [
    'Instance_Index',
    'STITCH_Code',
    'STITCH_Compound',
    'STITCH Code',
    'InteractorsList',
    'InteractorsCount'
]
cepi.drop(columns=remove, errors='ignore',inplace=True)
drpi.drop(columns=remove, errors='ignore',inplace=True)
cego.drop(columns=remove, errors='ignore',inplace=True)
drgo.drop(columns=remove, errors='ignore',inplace=True)

(cego.shape, drgo.shape, cepi.shape, drpi.shape)

((1120, 9001), (45, 9001), (1120, 5789), (45, 5789))

In [9]:
cepi.drop(columns=filtered_pi, errors='ignore',inplace=True)
drpi.drop(columns=filtered_pi, errors='ignore',inplace=True)
cego.drop(columns=filtered_go, errors='ignore',inplace=True)
drgo.drop(columns=filtered_go, errors='ignore',inplace=True)

(cego.shape, drgo.shape, cepi.shape, drpi.shape)

((1120, 7088), (45, 7088), (1120, 4990), (45, 4990))

In [133]:
summation = []
for feature in test_set:
    summation.append(test_set[feature].sum())
for i,x in enumerate(summation):
    if x==0:
        print(f'{test_set.columns[i]} = 0')

6239.F07B10.6 = 0
6239.F25C8.2a = 0
6239.R13G10.2 = 0
6239.T28F12.3 = 0
6239.B0280.12b = 0
6239.C06E1.4 = 0
6239.K10D3.1 = 0
6239.F33C8.1a = 0
6239.F16B4.8 = 0
6239.R05H5.2 = 0
6239.F02E8.6 = 0
6239.F09G8.4 = 0
6239.F32A11.2 = 0
6239.K10B3.10b = 0
6239.K11C4.3c = 0
6239.R31.1a = 0
6239.C44E4.6.1 = 0
6239.F44G4.8a = 0
6239.Y105E8A.20b = 0
6239.Y105E8A.28 = 0
6239.F38E11.3 = 0
6239.K07E3.8a = 0
6239.Y45F10A.4 = 0
6239.C54G4.6 = 0
6239.F21C3.3 = 0
6239.K02A11.3 = 0
6239.Y19D10A.16 = 0
6239.F55A11.11 = 0
6239.C47D2.2 = 0
6239.F49E8.4 = 0
6239.R03A10.3 = 0
6239.T04D3.3a = 0
6239.C09F5.2a = 0
6239.C47B2.6b = 0
6239.Y56A3A.29a = 0
6239.F47G4.6 = 0
6239.W02D9.3 = 0
6239.F47G4.3 = 0
6239.B0207.12b = 0
6239.F11A5.10 = 0
6239.H35N03.1 = 0
6239.R11G10.1a = 0
6239.C47A4.2b = 0
6239.F16H11.3 = 0
6239.F36H2.2a = 0
6239.F44D12.9c = 0
6239.K02E11.1 = 0
6239.ZK809.4a = 0
6239.F18A1.5 = 0
6239.H12C20.2a = 0
6239.K08H10.4 = 0
6239.R07B1.1 = 0
6239.F56B3.5 = 0
6239.T10F2.3 = 0
6239.H27M09.1 = 0
6239.R10E4.

In [139]:
training_set = aux.removeLowFrequencyFeatures(cepi, 10)
training_set.shape, test_set.shape

((1120, 5789), (45, 5789))

In [134]:
cepi.columns

Index(['6239.H13N06.6b', '6239.C41G7.9b', '6239.F15A8.5a', '6239.T07C4.8',
       '6239.Y39G8B.5', '6239.F07B10.5', '6239.F07B10.6', '6239.ZC482.5',
       '6239.C12C8.2a', '6239.ZK1127.10.1',
       ...
       '6239.T28F3.1a.2', '6239.F10G7.8.3', '6239.F35G12.12.1',
       '6239.F49C12.8.1', '6239.T12G3.1a.1', '6239.EEED8.9.2',
       '6239.F53H1.4a.1', '6239.C01F6.1.1', '6239.F52C12.6', 'Class'],
      dtype='object', length=5789)

In [2]:
import detectshift as ds
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

random_seed=0
np.random.seed(random_seed)

#DetectShift parameters
task='class'
test=.1 #test size fraction
B=500 #number of permutations

#Shift parameters
gamma=1
delta=0

In [10]:
drpi.iloc[:,:-1].values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [26]:
def GenData2(gamma, delta, n = 1000):
    ys = np.random.binomial(1, .5, n)  
    yt = np.random.binomial(1, .5 + delta, n)
    Xs=[]
    Xt=[]
    for i in range(n):
        Xs.append(np.random.normal(ys[i],1,3).tolist())
        Xt.append(np.random.normal(yt[i]+gamma,1,3).tolist())  
    Xs=np.array(Xs)
    Xt=np.array(Xt)
    Xs = Xs.reshape(n,-1)
    Xt = Xt.reshape(n,-1)
    ys = ys.reshape(-1)
    yt = yt.reshape(-1)
    return Xs, ys, Xt, yt

In [55]:
cepi[['Class']]

,Class
0,1
1,1
2,1
3,1
4,1
...,...
1115,0
1116,1
1117,1
1118,0


In [56]:
Xs, ys, Xt, yt =  cepi.iloc[:,:-1], cepi[['Class']], drpi.iloc[:,:-1], drpi[['Class']], 

In [57]:
Xs_train, Xs_test, ys_train, ys_test, Zs_train, Zs_test, \
Xt_train, Xt_test, yt_train, yt_test, Zt_train, Zt_test = ds.tools.prep_data(Xs, ys, Xt, yt, test=test, task=task, random_state=random_seed)    

In [58]:
#Training classifiers to estimate the R-N derivative
totshift_model = ds.tools.KL(boost=True)
totshift_model.fit(Zs_train, Zt_train)
covshift_model = ds.tools.KL(boost=True)
covshift_model.fit(Xs_train, Xt_train)

#Estimating the conditional distribution
cd_model = ds.cdist.cde_class(boost=True)
cd_model.fit(pd.concat([Xs_train, Xt_train], axis=0), 
             pd.concat([ys_train, yt_train], axis=0))

In [59]:
out = ds.tests.ShiftDiagnostics(Xs_test, ys_test, Xt_test, yt_test,
                                totshift_model=totshift_model, covshift_model=covshift_model, labshift_model=None,
                                cd_model=cd_model, task=task, B=B, verbose=True)

Calculating p-value for total shift...


100%|██████████| 500/500 [00:00<00:00, 26668.09it/s]



Calculating p-value for label shift...


  0%|          | 0/500 [00:00<?, ?it/s]c:\Users\Ewerton\Documents\github\auto_filter\.venv\Lib\site-packages\detectshift\tools.py:126: RuntimeWarning: divide by zero encountered in log
  kts+=pt*np.log(pt/ps)
c:\Users\Ewerton\Documents\github\auto_filter\.venv\Lib\site-packages\detectshift\tools.py:126: RuntimeWarning: invalid value encountered in scalar multiply
  kts+=pt*np.log(pt/ps)
100%|██████████| 500/500 [00:00<00:00, 12372.36it/s]



Calculating p-value for covariate shift...


...


Calculating p-value for concept shift type 1...


100%|██████████| 500/500 [02:19<00:00,  3.59it/s]



Calculating p-value for concept shift type 2...


  1%|          | 4/500 [00:03<06:29,  1.27it/s]


ValueError: Length mismatch: Expected axis has 5788 elements, new values have 5789 elements

In [31]:
pd.DataFrame(out).T.iloc[:,:2]

,pval,kl
tot,0.001996,1.322811
lab,0.750499,0.000804
cov,0.001996,0.79148
conc1,0.001996,1.322007
conc2,0.005988,0.531331


In [72]:
kts=0
ys = np.array([0,0,0,0,0,0,0])
yt = np.array([0,0,0,0,0,0,1])
Y=np.unique(ys)

for y in Y:
    pt=np.mean(np.array(yt).squeeze()==y)
    ps=np.mean(np.array(ys).squeeze()==y)
    kts+=pt*np.log(pt/ps)
    
kts 

-0.13212915413765

In [74]:
ps

1.0

In [3]:
cego.iloc[:,-1].value_counts()/len(cego)

Class
0    0.774107
1    0.225893
Name: count, dtype: float64

In [117]:

cepi.iloc[:,-1].value_counts()/len(cepi)


Class
0    0.774107
1    0.225893
Name: count, dtype: float64

In [118]:
drgo.iloc[:,-1].value_counts()/len(drgo)


Class
0    0.511111
1    0.488889
Name: count, dtype: float64

In [119]:
drpi.iloc[:,-1].value_counts()/len(drpi)

Class
0    0.511111
1    0.488889
Name: count, dtype: float64

In [91]:
cego.iloc[:,10].value_counts(ascending=True)[0]

1004

In [101]:
sorted(cego.iloc[:,10].value_counts())[0]

116

In [120]:
cepi.shape

(1120, 5794)

In [122]:
drpi.shape

(45, 5793)

In [123]:
drgo.shape

(45, 9005)

In [86]:
def get_0(df):
    count = 0
    for c in df.columns:
        if (df[c].sum() < 10):
            count += 1 
    return count

get_0(cego),get_0(cepi)

(1294, 0)

In [91]:
cego.shape

(1120, 9001)

In [95]:
from math import log2

# calculate the kl divergence
def kl_divergence(p, q):
    val = []
    min = 2**(-20)
    for i in range(len(p)):
        if p[i]==0:
            p[i]=min
        if q[i]==0:
            q[i]=min
        val.append(p[i] * np.log2(p[i]/q[i]))
    return sum(val)

In [93]:
p1 = cepi.iloc[:,0].value_counts().values/cepi.iloc[:,0].value_counts().values.sum()
p2 = drpi.iloc[:,0].value_counts().values/drpi.iloc[:,0].value_counts().values.sum()


In [96]:
df1_instances = cego.shape[0]
df2_instances = drgo.shape[0]
result=[]

for col in cego.columns[:-1]:
    p1 = cego[col].value_counts().reindex([0,1], fill_value=0).values/df1_instances
    p2 = drgo[col].value_counts().reindex([0,1], fill_value=0).values/df2_instances
    
    result.append([col, kl_divergence(p2,p1), kl_divergence(p1,p2)])

_test = pd.DataFrame(result, columns=['feature', '( p2 || p1 )', '( p1 || p2 )'])
_test.sort_values(by='( p2 || p1 )')

,feature,( p2 || p1 ),( p1 || p2 )
4603,GO:0000188,0.00000,0.000000
5248,GO:0071733,0.00000,0.000000
738,GO:0007257,0.00000,0.000000
2444,GO:0036425,0.00000,0.000000
4444,GO:2000582,0.00000,0.000000
...,...,...,...
2832,GO:0008584,0.35343,0.250840
7900,GO:0046546,0.35343,0.250840
7903,GO:0051728,0.39158,0.252578
7904,GO:0060184,0.39158,0.252578


In [100]:
for feature in _test[_test['( p2 || p1 )']==0]['feature']:
    if cego[feature].value_counts().values.shape[0] > 1:
        print(feature)

GO:0047748
GO:0047749


In [88]:
b = sorted(_test['( p2 || p1 )'].unique())
b

[0.0,
 3.020448485776899e-08,
 4.1066544693714154e-08,
 1.5370146335487358e-07,
 3.257879758792833e-07,
 5.526361208810313e-07,
 6.667760663271109e-07,
 1.02405345749968e-06,
 1.1525102627079871e-06,
 1.3988159442182265e-06,
 1.7923793011405441e-06,
 2.198904916983599e-06,
 2.2061947189820803e-06,
 4.130384469824009e-06,
 4.641842513521078e-06,
 4.871098597492626e-06,
 6.419746007021478e-06,
 6.564704264546556e-06,
 7.921254034582575e-06,
 8.273497487122725e-06,
 8.416536981270886e-06,
 8.891849857744312e-06,
 1.2230999897772012e-05,
 1.241421557806547e-05,
 1.3737122313645534e-05,
 1.3987058409021137e-05,
 1.4440994665443872e-05,
 1.4686738311292394e-05,
 1.5957331139100986e-05,
 1.6250895413795832e-05,
 1.7344936139032912e-05,
 1.8339049713922244e-05,
 1.8358259243477816e-05,
 1.9917198983294623e-05,
 2.0990901718929804e-05,
 2.141103877497199e-05,
 2.1596985877841632e-05,
 2.1770042201662253e-05,
 2.3724776167078666e-05,
 2.6032862789442373e-05,
 2.733560358255359e-05,
 3.1753927014

In [ ]:
cego['GO:0047748']

In [83]:
a = 'GO:0047749'
cego[a].value_counts(),drgo[a].value_counts()

(GO:0047749
 0    896
 1    224
 Name: count, dtype: int64,
 GO:0047749
 0    36
 1     9
 Name: count, dtype: int64)

In [36]:
p2.reshape(2,)

ValueError: cannot reshape array of size 1 into shape (2,)

In [ ]:

from sklearn.model_selection import StratifiedKFold  # maintains class proportions when creating folds


kf = StratifiedKFold(n_splits=10, shuffle=False)



In [25]:
cego.columns[2],drgo.columns[2]

('GO:0005507', 'GO:0005507')

In [46]:
cego.shape[0], cego['GO:0004500'].sum() 

(1120, 69)

In [50]:
drgo.shape[0], drgo['GO:0004500'].sum() 

(45, 1)

In [52]:
total = cego.shape[0]+drgo.shape[0]
positive = cego['GO:0004500'].sum() + drgo['GO:0004500'].sum()
positive/total, (total-positive)/total


(0.060085836909871244, 0.9399141630901288)

In [131]:
import scipy
import numpy as np
def myChi2(obs):
    f_exp = scipy.stats.contingency.expected_freq(obs.reshape(2,2)).ravel()
    chi = scipy.stats.chisquare(f_obs=obs, f_exp=f_exp)
    independency = scipy.stats.chi2_contingency(obs.reshape(2,2), correction=False).pvalue
    return chi, independency

In [134]:
df1_instances = cego.shape[0]
df2_instances = drgo.shape[0]
result=[]

for col in cego.columns[:-1]:
    positive1 = cego[col].sum()
    positive2 = drgo[col].sum()
    if ((positive1 > 0) & (positive1 < df1_instances) & (positive2 > 0) & (positive2 < df2_instances)):
        observed = np.array([[df1_instances-positive1, positive1],[df2_instances-positive2,positive2]])
        chi,independency=myChi2(observed.ravel())
        result.append([col, chi.statistic, chi.pvalue, independency])
_test = pd.DataFrame(result, columns=['feature', 'statistic', 'dependency', 'independency'])
_test.sort_values(by='statistic')

,feature,statistic,dependency,independency
1700,GO:0047749,0.000000,1.000000e+00,1.000000e+00
1699,GO:0047748,0.000000,1.000000e+00,1.000000e+00
684,GO:0007611,0.000002,1.000000e+00,9.989261e-01
4469,GO:0051051,0.000002,1.000000e+00,9.989261e-01
4174,GO:0046189,0.000002,1.000000e+00,9.987479e-01
...,...,...,...,...
2510,GO:0008584,31.246812,7.541708e-07,2.272203e-08
6244,GO:0046546,31.246812,7.541708e-07,2.272203e-08
6248,GO:0060184,38.395437,2.330857e-08,5.776730e-10
2524,GO:0051729,38.395437,2.330857e-08,5.776730e-10


In [137]:
_test[_test['independency']<0.01].sort_values(by='statistic')

,feature,statistic,dependency,independency
937,GO:0044325,6.635378,8.447374e-02,9.997300e-03
605,GO:0040002,6.641753,8.423666e-02,9.961587e-03
2183,GO:1990498,6.656814,8.367910e-02,9.877734e-03
1788,GO:0000387,6.656814,8.367910e-02,9.877734e-03
5848,GO:0016863,6.656814,8.367910e-02,9.877734e-03
...,...,...,...,...
2510,GO:0008584,31.246812,7.541708e-07,2.272203e-08
6244,GO:0046546,31.246812,7.541708e-07,2.272203e-08
2524,GO:0051729,38.395437,2.330857e-08,5.776730e-10
6247,GO:0051728,38.395437,2.330857e-08,5.776730e-10


In [63]:
import numpy as np
from scipy.stats.contingency import expected_freq
observed = np.array([[cego.shape[0]-cego['GO:0004500'].sum(), cego['GO:0004500'].sum()],[drgo.shape[0]-drgo['GO:0004500'].sum(), drgo['GO:0004500'].sum()]])
expected_freq(observed)



array([[1052.70386266,   67.29613734],
       [  42.29613734,    2.70386266]])

In [108]:
myChi2(observed.ravel())

Power_divergenceResult(statistic=1.1882402295012373, pvalue=0.7558261502558508)

In [110]:
chisquare(f_obs.reshape(2,2))

Power_divergenceResult(statistic=array([926.07214612,  66.05714286]), pvalue=array([2.10930069e-203, 4.38037542e-016]))

In [112]:
import scipy
scipy.stats.chi2_contingency(observed, correction=False)

Chi2ContingencyResult(statistic=1.1882402295012373, pvalue=0.2756848104299716, dof=1, expected_freq=array([[1052.70386266,   67.29613734],
       [  42.29613734,    2.70386266]]))

In [139]:
_one = pd.read_csv(r'C:\Users\Ewerton\Downloads\7227.protein_chemical.links.v5.0.tsv', sep='\t')

In [142]:
len(_one.chemical.unique()),len(_one.chemical),

(644408, 20087229)

In [97]:
np.array([[0,1,2],[3,4,5],[3,4,5]]).ndim

2

In [93]:
observed.size,sum(observed.shape),observed.ndim

(4, 4, 2)

In [90]:

from scipy.stats import chisquare

import numpy as np
f_exp = np.array([1052.7039, 67.2961, 42.2961, 2.7039])
f_obs = np.array([1051, 69, 44, 1])

chisquare(f_obs=f_obs, f_exp=f_exp)

Power_divergenceResult(statistic=1.1882775657657683, pvalue=0.7558171869270677)

In [17]:
f_exp

array([1989.610371,  127.189629,   79.939629,    5.110371])